# 1. Librerías & Set Up

👉🏻 Instalaremos todas las librerias necesarias para poder trabajar con PySpark y Neo4j

In [1]:
!pip install pyspark

In [2]:
!pip install neo4j

In [3]:
from pyspark.sql import SparkSession
from itertools import product

In [4]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
from itertools import combinations


In [5]:
# Iniciamos una sesion en PySpark
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# 2. Neo4j Graphs and others examples

👉🏻 Las siguientes funciones permiten cargar grafos de ejemplos para poder realizar las consultas de los patrones de grafo.

👉🏻 Existe la funcion create_neo4j_graph para poder cargar una base de datos en Neo4j y luego obtener las aristas existentes de este grafo.

👉🏻 Existe la funcion load_graph para poder seleccionar el grafo que queremos realizar las consultas.

In [6]:
def load_neo4j_graph():
  """
  Funcion para cargar la base de datos de twitter como ejemplo de grafo en Neo4j.
  """
  # Realizamos la conexion con la base de datos del grafo guardado en Neo4j.
  NEO4J_URI = "neo4j+s://demo.neo4jlabs.com"
  NEO4J_USERNAME = "twitter"
  NEO4J_PASSWORD = "twitter"
  AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

  driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
  with driver.session() as session:
      try:
          session.run("RETURN 1")
          print("Connection to Neo4j established successfully!")
      except Exception as e:
          print(f"Failed to connect to Neo4j: {e}")
      # Query que permite cargar todas las aristas de cada usuario
      # (descartamos el usuario Neo4j para tener una cantidad mas variada xd,
      # ya que si lo dejabamos se repetia mucho)
      query = """
      MATCH (u1:User)-[r:INTERACTS_WITH | SIMILAR_TO]->(u2:User)
      WHERE u1.name <> "Neo4j" AND u2.name <> "Neo4j"
      RETURN u1.name as user1, type(r) AS relationship, u2.name AS user2
      """
      result = session.run(query)
      df_edges = pd.DataFrame(result.data())
      relationship = df_edges['relationship'].unique()

      # Definimos una clase para cada relacion ...
      relationship_class = {i : rel for rel, i in enumerate(relationship)}
      for inter, inter_class in relationship_class.items():
          print(f"{inter} AS class {inter_class}")
      df_edges['relationship'].replace(relationship_class, inplace = True)

      # Transformamos cada usuario a un numero (su identificador) en base
      # a su nombre de usuari que debe ser unico ...
      users1 = df_edges['user1'].unique().tolist()
      users2 = df_edges['user2'].unique().tolist()
      users = list(set(users1 + users2))
      users_id = {user : id for id, user in enumerate(users)}
      df_edges['user1'].replace(users_id, inplace = True)
      df_edges['user2'].replace(users_id, inplace = True)

      np.save('users_id', users_id, allow_pickle = True)
      np.save('relationship_class', relationship_class, allow_pickle = True)

      # Cargamos el grafo en una lista de tuplas
      neo4f_graph = [(item[0], item[1], item[2]) for item in df_edges.values]
  driver.close()
  return neo4f_graph


In [7]:
def load_graph(option):
  """
  Funcion que selecciona el grafo de ejemplo, retorna una lista de tuplas
  de las aristas del grafo.
  """
  if option == "neo4j-graph":
    neo4f_graph = load_neo4j_graph()
    return neo4f_graph

  elif option == "basic-example":
    return [
        (1,11,2), (1,11,3), (2,11,3), (3,11,2),
        (3,11,4), (4,11,1), (4,11,2), (4,11,3),
        (4,12,5), (5,12,1), (5,12,2), (5,12,6),
        (1,12,2), (2,12,3), (3,12,1)
    ]
  else:
    print("La opcion no existe :(")
    return None

# 3. MapReduce Algorithm

## 3.1 Implementation for triangles

👉🏻 Las siguientes funciones permiten desarrollar el algoritmo MapReduce implementado con PySpark

In [8]:
def hash(n):
  """
  Returns number mod 2. The ouput will be 0 or 1.
  """
  return n % 2

👉🏻 Las funciones siguientes permiten simular la fase de mapeo del algoritmo.

In [9]:
def get_keys(edge, b_dim, b_set, pattern_dim):
  """
  output: retorna las llaves correspondientes para un vertice.

  Idea general: buscamos el par hash_n1,hash_n2 dentro de las posibles
  combinaciones dentro del espacio de imagenes de la funcion de hash.
  Dentro del for, obtenemos un string con la codificacion de las llaves y luego
  verificamos si es una llave candidata para el vertice entregado:

  1.  sequence_in_reducer: Si la secuencia 'b1b2' esta en la llave del reducer
      codificada como hash(n1)hash(n2) ?, donde ? = 0 o 1, entonces se considerará
      el par reducer_key : edge.
  2.  edge_case: El otro caso, es para cuando tenemos por ejemplo x = n2 y z = n1
      para patrones de 3 vertices.
  """

  hash_n1 = hash(edge[0]) # valor de hash para el nodo 1 = b1
  hash_n2 = hash(edge[2]) # valor de hash para el nodo 2 = b2
  values = [] # posible keys
  sequence = '{}{}'.format(hash_n1, hash_n2)

  for i in range(0, b_dim ** pattern_dim):
    reducer = ''.join(str(num) for num in b_set[i])
    sequence_in_reducer = sequence in reducer
    edge_case = reducer[0] == sequence[1] and reducer[pattern_dim - 1] == sequence[0]
    if sequence_in_reducer or edge_case:
        reducer_key = tuple(int(digit) for digit in reducer)
        values.append((reducer_key, edge))

  return values


def map_phase(rdd, b_dim, b_set, pattern_dim):
  """
  input:
    - rdd: RDD del grafo de dimension 'dim'
    - b_dim: Cantidad de elementos de las imagenes de la funcion de hash.
    - b_set: Imagenes de la funcion de hash.
    - pattern_dim: cantidad de nodos del patron de grafo.
  ouput: Mapeo de cada arista con respecto a las llaves
  """

  mapped_keys = rdd.flatMap(lambda edge: get_keys(edge, b_dim, b_set, pattern_dim))

  return mapped_keys

👉🏻 Las siguientes funciones permiten simular la fase de reduce del algoritmo.

In [10]:
def find_patterns(edges, pattern_dim):
    """
    Funcion que retorna todos los patrones encontrados de la forma
    (n1,...,nl) : [[(n1,label,n2), ..., (nl-1,label,nl)], ... ]
    donde (n1,...,nl) corresponde a los nodos que forman el patron
    y la llave corresponde a los patrones posibles formados por estos nodos
    para una etique
    """
    neighbors = get_neighbors(edges)
    cycles = find_cycles(edges, neighbors, pattern_dim)
    return cycles


def get_neighbors(edges):
    """
    Funcion que retorna todas las aristas consecutivas a otras aristas.
    Es decir, aristas vecinas.
    """
    neighbors = []
    for i in range(len(edges)):
        curr_edge = edges[i]
        neighbors.append([])
        for j in range(len(edges)):
            next_edge = edges[j]
            if next_edge == curr_edge:
                continue
            if curr_edge[2] == next_edge[0]:
                neighbors[i].append(next_edge)
    return neighbors


def find_cycles(edges, neighbors, pattern_dim):
    """
    Funcion que verifica si existe algun grafo ciclico en el grafo de una
    dimension pattern_dim. Retorna todos los ciclos encontrados.
    Utilizamos el algoritmo DFS iterativo para la busqueda de ciclos.
    """
    cycles = [] # Lista de ciclos encontrados

    for edge in edges: # Por cada arista del grafo
        visited = set() # Determinamos las aritas que ya han sido visitadas
        # Inicializamos el stack con con la
        stack = [(edge, [edge])]
        # arista donde empezamos el recorrido

        while len(stack):
            curr_edge, path = stack.pop() # Extraemos la primera arista del stack
            # Recorremos esta arista, por lo que la marcamos como visitada
            visited.add(curr_edge)
            # Caso 1: Si el camino recorrido actual supera la cantidad de nodos del bgp,
            #         descartamos este camino
            if len(path) > pattern_dim:
                continue
            # Caso 2: Si el camino recorrido tiene un largo igual a pattern_dim,
            #         y son aristas transitivas, entonces un posible ciclo.
            first_node = path[0][0] # primer nodo del camino
            last_node = path[-1][2] # ultimo nodo del caminmo
            if len(path) == pattern_dim and first_node == last_node:
                cycles.append(path)
                # Agregamos el camino y seguimos recorriendo
                continue
            # Si no es ciclico el camino desde el nodo actual,
            # entonces iteramos dentro de sus vecinos
            for neighbor_edge in neighbors[edges.index(curr_edge)]:
                if neighbor_edge not in visited:
                    stack.append((neighbor_edge, path + [neighbor_edge]))

    return cycles


def get_nodes(pattern, pattern_dim):
    """
    Funcion que retorna la tupla con los nodos que forman un ciclo.
    """
    nodes = []
    for i in range(pattern_dim):
      edge = pattern[i]
      nodes.append(edge[0])

    return (tuple(sorted(nodes)), tuple(sorted(pattern)))


def get_unique_lists(pattern1, pattern2):
    """
    Funcion que elimina patrones de grafo duplicados para cada llave
    (n1,n2, ..., nl). Si tenemos l = 3, entonces seria (n1,n2,n3)
    """
    lst = []
    if pattern2 != pattern1:
      return lst.append(pattern2)
    else:
      return pattern1


def reduce_phase(mapped_keys, pattern_dim):
    """
    input: RDD del grafo y cantidad de nodos del patron de grafo.
    output: patrones encontrados.
    """
    # Agrupamos por llaves
    reducers = mapped_keys.groupByKey().mapValues(list)
    # Filtramos solamente los grupos conformados por 3 aristas ...
    reducers_edges = reducers.map(lambda v: v[1]) \
                    .filter(lambda x: len(x) >= pattern_dim)
    # Por cada grupo buscamos un patron de grafo
    patterns = reducers_edges \
              .map(lambda edges: find_patterns(edges, pattern_dim)) \
              .flatMap(list) \
              .map(lambda pattern: get_nodes(pattern, pattern_dim)) \
              .groupByKey().mapValues(lambda v: list(set(v)))


    return patterns

👉🏻 Funcion final que simula el algoritmo MapReduce. Recibe los parametros correspondientes del grafo; las dimensiones del conjunto de imagenes de la funcion de hash; el conjunto de imagenes de la funcion de hash; la dimension del patron de grafo.

In [11]:
def map_reduce(rdd_graph, b_dim, b_set, pattern_dim, show=False):
  """
  Funcion que simula el algoritmo MapReduce, en donde se distribuye el grafo
  en diferentes reducers y luego obtenemos los posibles patrones
  de grafos formados combinando las informacion de todos los reducers.
  """
  # Fase de Map: Obtenemos las llaves de cada reducer y el conjunto de aristas mapeados a estas llaves.
  mapped_keys = map_phase(rdd_graph, b_dim, b_set, pattern_dim)

  if show: # Mostras las llaves mapeadas
    print("Fase Map: \n")
    for key in mapped_keys.collect():
      print(key)
    print("\nFase Reduce\n")

  # Fase Reduce: Obtenemos todos los posibles patrones de L nodos.
  patterns = reduce_phase(mapped_keys, pattern_dim)
  return patterns

## 3.2 Implementation for Squares




👉🏻 Las siguientes funciones fueron definidas para realizar las consultas de sobre subgrafos de 4 variables.

In [61]:
def find_subgraphs(edges):
    """
    Funcion que retorna todos los posibles subgrafos de 4 nodos que existen
    dentro de un subgrafo de cada reducer.
    """

    size = len(edges)
    all_patterns = []

    for edge in edges: # iteramos por cada arista
        pattern = [edge] # inicializamos un posible patron de solo 4 nodos
        initial_edge = edge
        # inicializamos los nodos del subgrafo
        nodes = set([initial_edge[0], initial_edge[2]])

        for curr_edge in edges: # iteramos por el resto de las aristas
            if initial_edge == curr_edge:
                continue
            # consideramos los nodos que ya estan dentro del subgrafo
            # mas el nuevo nodo
            new_nodes = nodes.union([curr_edge[0], curr_edge[2]])

            # si al agregar el nuevo nodo no superamos las 4 variables,
            # entonces lo agregamos (junto con su arista respectiva)
            if len(new_nodes) <= 4:
                pattern.append(curr_edge)
                nodes = new_nodes # consideramos el nuevo conjunto de nodos

        if len(nodes) == 4:
            all_patterns.append(pattern)

    return all_patterns

def get_all_combinations(edges):
    """
    Función que retorna todos los posibles subgrafos de 4 nodos que existen
    dentro de un conjunto de aristas.
    """
    all_patterns = []
    # busqueda de subgrafos de solo 4 variables
    all_patterns = find_subgraphs(edges)
    # DFS para encontrar grafos acicliclos usando la funcion find_patterns
    neighbors = get_neighbors(edges)
    cycles = find_cycles(edges, neighbors, pattern_dim=4)
    all_patterns.extend(cycles)

    return all_patterns

def map_reduce_squares(rdd_graph, b_dim, b_set, pattern_dim):
    """
    Funcion que simula el algoritmo Map Reduce para subgrafos de solo 4 nodos.
    Retorna todos los posibles subgrafos del grafo entregado en la RDD
    con solo 4 nodos.
    """
    # Realizamos el Map Phase para obtener todos los pares llaves - valor
    mapped_keys = map_phase(rdd_graph, b_dim, b_set, pattern_dim)
    # Reducimos los valores dentro de cada reducer, donde solo consideramos
    # algun grupo que tenga al menos 3 aristas (posible patron de 4 variables)
    reducers = mapped_keys.groupByKey().mapValues(list)
    # Dentro de cada reducer, formamos todos los posibles patrones de solo
    # 4 variables distintas
    all_combinations = reducers.mapValues(get_all_combinations)
    # filtramos las llaves que tienen como valores nulos (subgrafos con dimension menor a 4 )
    all_combinations = all_combinations.filter(lambda x: x[1] != [])

    #return all_combinations
    return all_combinations


def preprocess_query(query):
    """
    Funcion que retorna la consulta en una lista de tuplas
    ingresada en get_all_matches.
    """
    processed_query = query.replace(" ", ";").replace("(", "").replace(")", "").split(",;")
    processed_query = [tuple(item.split(",")) for item in processed_query]
    return processed_query


def create_matrix_query(preprocessed_query):
    """
    Funcion que retorna la matriz de consulta.
    A: variables
    L: labels
    """
    A, L = [], []
    for edge in preprocessed_query:
        if edge[0] not in A:
            A.append(edge[0])
        if edge[2] not in A:
            A.append(edge[2])
        if int(edge[1]) not in L:
            L.append(int(edge[1]))

    # asociamos cada variable/label un indice
    dict_var_dir = {var: dir for dir, var in enumerate(A)}
    dict_label_dir = {label: dir for dir, label in enumerate(L)}
    M = np.zeros((len(A), len(L), len(A))) # matriz de consulta de dimension |A|x|L|x|A|
    # Rellenamos la matriz M con solo 1's en las posiciones donde exista una arista
    for edge in preprocessed_query:
        node_1 = edge[0] # primer nodo n1
        label = int(edge[1]) # label n1 -[label]-> n2
        node_2 = edge[2] # segundo nodo n2
        dir_node_1 = dict_var_dir[node_1]
        dir_label = dict_label_dir[label]
        dir_node_2 = dict_var_dir[node_2]

        M[dir_node_1][dir_label][dir_node_2] = 1

    return M

def matches(subgraphs, matrix_query):
    """
    Funcion que retorna los subgrafos que realizan match con
    la matriz de consulta (patron de grafo)
    """
    results = []
    # Iteramos por todo el subrgrafo y verificamos las posibles contenidas
    # que hagan match con la matriz de consulta
    for subgraph in subgraphs: # para cada subgrafo de la llabe
        match_found = False
        for i in range(len(subgraph)):
            if match_found:
                break
            # hacemos un desplazamiento de aristas
            shift_subgraph = subgraph[i:] + subgraph[:i]
            # obtenemos la matriz correspondiente a este subgrafo
            matrix_subgraph = create_matrix_query(shift_subgraph)
            # si hacen match, entonces lo agregamos y seguimos viendo otro subgrafo
            if np.array_equal(matrix_query, matrix_subgraph):
                results.append(subgraph)
                match_found = True

    return results

def generate_nodes(subgraph):
    """
    Funcion que retorna una tupla de los nodos de un subgrafo.
    """
    return tuple(set(edge[0] for edge in subgraph).union(edge[2] for edge in subgraph))


def get_all_matches(rdd_graph, query, show=False, more_info=False):
    """
    Funcion que retorna todos los matches con respecto a una consulta de
    solo 4 variables
    """
    # Primero obtenemos todos los posibles subgrafos de 4 nodos, independiente
    # de la cantidad de aristas
    b_sq = 2
    l_sq = 4
    b_set_0 = list(range(b_sq))
    reducers_sq = list(product(b_set_0, repeat=l_sq))

    # obtenemos todos los subgrafos de 4 nodos con MapReduce
    all_subgraphs = map_reduce_squares(rdd_graph, b_sq, reducers_sq, l_sq)
    preprocessed_query = preprocess_query(query)
    # matriz de consulta codificada
    matrix_query = create_matrix_query(preprocessed_query)
    # todos los patrones de grafos que hacen match
    patterns_result = all_subgraphs \
                        .mapValues(lambda values: matches(values, matrix_query))\
                        .filter(lambda v: v[1] != []).values().flatMap(list)
    if more_info:
        return patterns_result.map(lambda x: (generate_nodes(x), x))

    result = patterns_result.map(lambda x: generate_nodes(x)).distinct()

    if show:
        for k, vs in all_subgraphs.collect():
            for v in vs:
                print(v)

    return result.collect()

# 4. Aplicacion MapReduce en Ejemplo Basico

👉🏻 En esta seccion aplicaremos el algoritmo MapReduce para un grafo de prueba pequeño (el que fue dado al inicio de la tarea, con algunas modificaciones). Este lo guardamos como 'basic-example'.

In [13]:
graph = load_graph('basic-example')
rdd_graph = sc.parallelize(graph)
rdd_graph.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6),
 (1, 12, 2),
 (2, 12, 3),
 (3, 12, 1)]

## 4.1 MapReduce Algorithm for Triangles

👉🏻 Para encontrar triangulos dentro de un grafo utilizaremos el algoritmo Map Reduce para parametros b_0 = 2 y l = 3.

👉🏻 En este caso, b_0=2 ya que al utilizar la funcion de hash modular `f(x) = x mod 2`, generamos un conjunto de 2 imagenes: {0,1}. En consecuencia, los posibles reducers son:
```markdown
REDUCERS = [(0, 0, 0),
            (0, 0, 1),
            (0, 1, 0),
            (0, 1, 1),
            (1, 0, 0),
            (1, 0, 1),
            (1, 1, 0),
            (1, 1, 1)]
```

👉🏻 Dado que estamos buscando subgrafos de 3 nodos, entonces l_0 = 3.

👉🏻 Las aristas se distribuyen dentro de los reducers, por lo que almacenamos el grafo de manera distribuida.En cada reducer verificamos si las aristas generan triangulos, en el caso que si (formaria un ciclo) se almacenará en una lista y retornaremos los nodos correspondientes.




In [14]:
# Dimension de elementos del conjunto de imagenes de la funcion de hash: |{0,1}|
b_0 = 2
# Dimension del patron de grafo (triangulo para este caso)
l_0 = 3
# Conjunto de imagenes de la funcion de hash
b_set_0 = list(range(b_0))
# Posibles reducers
reducers_0 = list(product(b_set_0, repeat=l_0))

patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)

patterns.collect()

[((2, 3, 4),
  [((2, 11, 3), (3, 11, 4), (4, 11, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 11, 2))]),
 ((1, 2, 3),
  [((1, 12, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 12, 1))]),
 ((1, 3, 4), [((1, 11, 3), (3, 11, 4), (4, 11, 1))])]

In [15]:
patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0, show=True)

patterns.collect()

Fase Map: 

((0, 0, 1), (1, 11, 2))
((0, 1, 0), (1, 11, 2))
((0, 1, 1), (1, 11, 2))
((1, 0, 0), (1, 11, 2))
((1, 0, 1), (1, 11, 2))
((1, 1, 0), (1, 11, 2))
((0, 1, 1), (1, 11, 3))
((1, 0, 1), (1, 11, 3))
((1, 1, 0), (1, 11, 3))
((1, 1, 1), (1, 11, 3))
((0, 0, 1), (2, 11, 3))
((0, 1, 0), (2, 11, 3))
((0, 1, 1), (2, 11, 3))
((1, 0, 0), (2, 11, 3))
((1, 0, 1), (2, 11, 3))
((1, 1, 0), (2, 11, 3))
((0, 0, 1), (3, 11, 2))
((0, 1, 0), (3, 11, 2))
((0, 1, 1), (3, 11, 2))
((1, 0, 0), (3, 11, 2))
((1, 0, 1), (3, 11, 2))
((1, 1, 0), (3, 11, 2))
((0, 0, 1), (3, 11, 4))
((0, 1, 0), (3, 11, 4))
((0, 1, 1), (3, 11, 4))
((1, 0, 0), (3, 11, 4))
((1, 0, 1), (3, 11, 4))
((1, 1, 0), (3, 11, 4))
((0, 0, 1), (4, 11, 1))
((0, 1, 0), (4, 11, 1))
((0, 1, 1), (4, 11, 1))
((1, 0, 0), (4, 11, 1))
((1, 0, 1), (4, 11, 1))
((1, 1, 0), (4, 11, 1))
((0, 0, 0), (4, 11, 2))
((0, 0, 1), (4, 11, 2))
((0, 1, 0), (4, 11, 2))
((1, 0, 0), (4, 11, 2))
((0, 0, 1), (4, 11, 3))
((0, 1, 0), (4, 11, 3))
((0, 1, 1), (4, 11, 3))
((1,

[((2, 3, 4),
  [((2, 11, 3), (3, 11, 4), (4, 11, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 11, 2))]),
 ((1, 2, 3),
  [((1, 12, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 12, 1))]),
 ((1, 3, 4), [((1, 11, 3), (3, 11, 4), (4, 11, 1))])]

👉🏻 Al aplicar el algoritmo mapReduce, podemos notar que existen 3 tripletas de nodos que forman un triangulo. En donde estos nodos permiten formar diferentes patrones de grafos triangulares. En este caso, como tenemos mas de una etiqueta distinta, podemos notar que para una tripleta de nodo, forman mas de un triangulo.
```markdown
  (2,3,4); (1,2,3); (1,3,4)
```

👉🏻 Las siguientes funciones se definiran con el fin de desarrollar consultas sobre estos subgrafos triangulares...


In [16]:
def filter_patterns(pattern, pattern_dim, labels):
  """
  Funcion que retorna los patrones filtrados segun el label proporcionado como
  parametro.
  """
  pattern_labels = [edge[1] for edge in pattern]
  for i in range(pattern_dim + 1):
      # verificamos si coinciden los labels ...
      if pattern_labels == labels:
        return pattern
      # Hacemos un desplazamiento hacia la derecha ...
      pattern_labels = pattern_labels[i:] + pattern_labels[:i]

  return None

def bgp_query(rdd_graph, b_dim, reducers, pattern_dim, query):
  """
  Funcion que retorna todos los patrones de grafos de dimension "pattern_dim"
  que coincidan segun la query entregada.
  """

  patterns = map_reduce(rdd_graph, b_dim, reducers, pattern_dim).values().flatMap(list)
  processed_query = query.replace("(", "").replace(")", "").split(",")
  labels = [int(part.strip()) for part in processed_query if part.strip().isdigit()]

  result = patterns.filter(lambda pattern: filter_patterns(pattern, pattern_dim, labels))
  return result

In [17]:
# Realizamos la siguiente consulta en donde queremos obtener todos los triangulos
# donde la etiqueta es 11, independiente del nodo.
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,11,y), (y,11,z), (z,11,w)")
for res in result.collect():
  print(res)

((2, 11, 3), (3, 11, 4), (4, 11, 2))
((1, 11, 3), (3, 11, 4), (4, 11, 1))


## 4.2 MapReduce Algorithm for Squares


👉🏻 Ahora aplicaremos nuestro algoritmo para la busqueda de patrones de grafos de 4 variables. En primer lugar, utilizaremos el algoritmo Map Reduce definido para patrones de grafo de solo 4 variables definido en la **seccion 3.2**.
Esta funcion utiliza retorna todos los patrones de 4 variables que hagan match con la consulta definida en la funcion.

In [18]:
# A continuacion mostraremos todos los subgrafos posibles de solo 4 nodos,
# independiente de las aristas que tengan
get_all_matches(rdd_graph, "(x,12,y), (z,11,w), (x,12,z), (x,12,w), (z,12,w)")

[(1, 2, 5, 6)]

In [19]:
# Otro ejemplo para encontrar grafos cicliclos de solo 4 variables
get_all_matches(rdd_graph, "(x,11,y), (y,11,z), (z,11,w), (w,11,x)")

[(1, 2, 3, 4)]

In [20]:
get_all_matches(rdd_graph, "(x,11,y), (y,11,z), (z,12,w), (w,12,x)")

[(2, 3, 4, 5), (1, 2, 3, 4), (1, 3, 4, 5)]

# 5. Aplicacion de un grafo a gran escala: Twitter Graph

👉🏻 Ahora cargaremos un grafo de mayor tamaño (700 aristas) para evaluar nuestro algoritmo. El siguiente grafo corresponde a un grafo que modela la interaccion de cada usuario en la red social twitter.
El grafo sera cargado como lista de tuplas, en donde cada tupla modela la relacion entre un usuario y otro. Consideramos las siguientes 2 tipos de relaciones

    Relacion 0: INTERACTS_WITH - El usuario 1 interacciona con otro usuario 2
    Relacion 1: SIMILAR_TO - El usuario 1 es similar al usuario 2

Referencia: https://github.com/neo4j-graph-examples/twitter-v2


In [42]:
graph = load_graph('neo4j-graph')
rdd_graph = sc.parallelize(graph)

Connection to Neo4j established successfully!
INTERACTS_WITH AS class 0
SIMILAR_TO AS class 1


In [43]:
rdd_graph.count() # Cantidad de aristas del grafo

744

## 5.1 MapReduce Algorithm for Triangles

👉🏻 Ahora aplicaremos MapReduce para encontrar los patrones de grafos con forma de triangulo dentro del grafo cargado desde Neo4j.



In [44]:
patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)
patterns.collect()

[((275, 302, 454), [((275, 1, 302), (302, 0, 454), (454, 1, 275))]),
 ((68, 302, 505), [((68, 1, 302), (302, 0, 505), (505, 1, 68))]),
 ((281, 292, 412), [((281, 0, 292), (292, 1, 412), (412, 0, 281))]),
 ((292, 412, 479), [((292, 1, 412), (412, 0, 479), (479, 1, 292))]),
 ((68, 275, 454), [((68, 0, 454), (275, 1, 68), (454, 1, 275))]),
 ((29, 302, 458),
  [((29, 1, 458), (302, 0, 29), (458, 1, 302)),
   ((29, 1, 302), (302, 0, 458), (458, 0, 29))]),
 ((147, 390, 464),
  [((147, 1, 390), (390, 0, 464), (464, 0, 147)),
   ((147, 1, 464), (390, 0, 147), (464, 1, 390))]),
 ((4, 281, 350), [((4, 1, 350), (281, 0, 4), (350, 1, 281))]),
 ((4, 350, 479), [((4, 1, 350), (350, 1, 479), (479, 0, 4))]),
 ((4, 317, 350), [((4, 1, 350), (317, 0, 4), (350, 1, 317))]),
 ((317, 344, 350),
  [((317, 0, 344), (344, 0, 350), (350, 1, 317)),
   ((317, 0, 350), (344, 1, 317), (350, 1, 344))]),
 ((163, 344, 350), [((163, 0, 350), (344, 0, 163), (350, 1, 344))]),
 ((4, 344, 479), [((4, 1, 344), (344, 1, 479)

👉🏻 El algoritmo es bien rapido de ejecutar, y podemos ver claramente las relaciones entre diferentes usuarios, para diferentes labels.

👉🏻 Para poder analizar las relaciones y compararlo con el grafo mostrado en Neo4j, realizaremos una consulta, y luego compararemos los resultados ...

In [45]:
# Cargamos el diccionario que guardamos los nombres de usuarios y el id asociado.
users_id = np.load('users_id.npy', allow_pickle=True).item()
dict_users_id = {value : key for key, value in users_id.items()}
# Cargamos el tipo de relacion.
relationship_class = np.load('relationship_class.npy', allow_pickle=True).item()
dict_relation_class = {value : key for key, value in relationship_class.items()}

In [46]:
# Vamos hacer la siguiente consulta para un patron de grafo de 3 aristas
# y luego reflejaremos los usuarios involucrados ...
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,1,y), (y,0,z), (z,0,x)")
result.collect()

[((281, 0, 292), (292, 1, 412), (412, 0, 281)),
 ((29, 1, 302), (302, 0, 458), (458, 0, 29)),
 ((147, 1, 390), (390, 0, 464), (464, 0, 147)),
 ((4, 1, 163), (163, 0, 350), (350, 0, 4)),
 ((4, 1, 317), (317, 0, 344), (344, 0, 4)),
 ((281, 0, 479), (351, 1, 281), (479, 0, 351)),
 ((145, 1, 317), (163, 0, 145), (317, 0, 163)),
 ((74, 1, 272), (152, 0, 74), (272, 0, 152)),
 ((281, 0, 479), (344, 1, 281), (479, 0, 344)),
 ((4, 1, 317), (163, 0, 4), (317, 0, 163)),
 ((145, 1, 281), (281, 0, 344), (344, 0, 145)),
 ((145, 1, 317), (317, 0, 344), (344, 0, 145)),
 ((163, 1, 479), (344, 0, 163), (479, 0, 344)),
 ((163, 1, 281), (281, 0, 344), (344, 0, 163)),
 ((4, 1, 163), (145, 0, 4), (163, 0, 145)),
 ((145, 1, 479), (344, 0, 145), (479, 0, 344)),
 ((4, 1, 317), (145, 0, 4), (317, 0, 145))]

In [47]:
def load_usernames(pat):
  """
  Funcion que retorna el nombre de usuario codificado a un id, y tambien transforma
  la clase del label a su valor original
  """
  list_users = []
  for i in range(len(pat)):
    edge = pat[i]
    user1 = dict_users_id[edge[0]]
    label = dict_relation_class[edge[1]]
    user2 = dict_users_id[edge[2]]
    list_users.append((user1, label, user2))
  return list_users

👉🏻 Ahora con la consulta anterior, analizaremos los usuarios involucrados en el siguiente caso: el primer usuario interacciona con el segundo; el segundo es similar al tercero; y el tercero el similar al primero.

In [48]:
result.map(load_usernames).collect()

[[('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs',
   'INTERACTS_WITH',
   'Chidambara .ML.'),
  ('Chidambara .ML.', 'SIMILAR_TO', 'Lju Lazarevic'),
  ('Lju Lazarevic',
   'INTERACTS_WITH',
   'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs')],
 [('Nigel Small 🇪🇺', 'SIMILAR_TO', 'Michael Simons'),
  ('Michael Simons', 'INTERACTS_WITH', 'Jim Webber'),
  ('Jim Webber', 'INTERACTS_WITH', 'Nigel Small 🇪🇺')],
 [('Mahendrawathi ER', 'SIMILAR_TO', 'Nur Aini Rakhmawati'),
  ('Nur Aini Rakhmawati', 'INTERACTS_WITH', 'Radit - Pendekar Syair Berbusa'),
  ('Radit - Pendekar Syair Berbusa', 'INTERACTS_WITH', 'Mahendrawathi ER')],
 [('guinnessstache', 'SIMILAR_TO', 'Chill Zone'),
  ('Chill Zone', 'INTERACTS_WITH', 'Wen.S'),
  ('Wen.S', 'INTERACTS_WITH', 'guinnessstache')],
 [('guinnessstache', 'SIMILAR_TO', 'Born To Be Idle'),
  ('Born To Be Idle', 'INTERACTS_WITH', 'John Wolpert'),
  ('John Wolpert', 'INTERACTS_WITH', 'guinnessstache')],
 [('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs',
   'INTERACTS_WITH',
   

## 5.2 MapReduce For Squares

👉🏻 Ahora aplicaremos el algoritmo MapReduce para buscar patrones todos los posibles subgrafos de solo 4 variables, para luego realizar una consulta.

In [49]:
get_all_matches(rdd_graph, "(x,1,y), (z,0,w), (y,0,x)")

[(361, 2, 29, 302),
 (361, 2, 505, 302),
 (361, 2, 302, 215),
 (361, 2, 412, 85),
 (361, 2, 412, 281),
 (361, 2, 412, 181),
 (361, 2, 412, 479),
 (81, 361, 2, 412),
 (361, 2, 68, 505),
 (361, 458, 2, 29),
 (361, 194, 2, 135),
 (361, 2, 286, 55),
 (329, 361, 450, 2),
 (329, 450, 2, 361),
 (361, 2, 147, 390),
 (2, 361, 34, 169),
 (281, 2, 292, 361),
 (281, 2, 206, 361),
 (344, 281, 2, 361),
 (281, 2, 412, 361),
 (361, 2, 206, 281),
 (169, 34, 2, 361),
 (361, 2, 292, 281),
 (361, 2, 292, 479),
 (288, 353, 2, 361),
 (288, 361, 2, 353),
 (361, 2, 270, 295),
 (344, 361, 2, 479),
 (361, 2, 4, 317),
 (361, 2, 317, 350),
 (344, 361, 2, 317),
 (344, 361, 2, 163),
 (344, 361, 145, 2),
 (344, 361, 2, 71),
 (344, 361, 2, 281),
 (361, 2, 163, 4),
 (361, 2, 163, 350),
 (145, 2, 361, 350),
 (145, 2, 4, 361),
 (344, 145, 2, 361),
 (361, 145, 2, 350),
 (361, 145, 2, 4),
 (472, 361, 2, 351),
 (464, 361, 2, 147),
 (361, 2, 275, 454),
 (302, 505, 2, 361),
 (505, 2, 68, 361),
 (81, 2, 412, 361),
 (280, 68, 

In [50]:
get_all_matches(rdd_graph, "(x,0,y), (w,0,z), (w,0,x), (x,1,w)")

[]

In [51]:
get_all_matches(rdd_graph, "(x,0,y), (y,0,z), (z,0,w), (w,1,x)")

[(412, 458, 68, 302),
 (412, 68, 454, 302),
 (281, 292, 412, 479),
 (281, 412, 206, 479),
 (4, 317, 350, 479),
 (163, 4, 350, 479),
 (145, 4, 350, 479),
 (302, 412, 68, 454),
 (344, 281, 292, 479),
 (344, 281, 163, 4),
 (412, 292, 281, 479),
 (344, 281, 206, 479),
 (344, 163, 4, 479),
 (344, 163, 4, 317),
 (344, 145, 317, 350),
 (344, 163, 317, 350),
 (344, 281, 163, 350),
 (344, 163, 350, 479),
 (344, 145, 4, 479),
 (344, 145, 4, 317),
 (344, 145, 4, 281),
 (344, 281, 145, 350),
 (505, 68, 302, 412),
 (281, 412, 68, 76),
 (412, 68, 505, 302),
 (281, 292, 351, 479),
 (344, 281, 4, 350),
 (472, 281, 351, 479),
 (281, 76, 412, 68),
 (344, 4, 350, 479),
 (344, 4, 317, 350),
 (344, 145, 163, 317),
 (344, 145, 163, 281),
 (302, 505, 68, 412),
 (412, 76, 281, 68),
 (163, 4, 317, 479)]

In [52]:
get_all_matches(rdd_graph, "(x,0,y), (z,0,w)")

[(361, 2, 445, 302),
 (361, 2, 441, 302),
 (361, 2, 193, 302),
 (361, 2, 302, 439),
 (361, 2, 302, 273),
 (361, 2, 302, 199),
 (361, 2, 123, 302),
 (361, 2, 115, 302),
 (361, 2, 302, 247),
 (361, 2, 302, 231),
 (361, 137, 2, 302),
 (361, 2, 412, 481),
 (361, 153, 2, 412),
 (361, 2, 443, 412),
 (361, 2, 412, 37),
 (361, 2, 412, 351),
 (361, 2, 260, 297),
 (361, 2, 475, 68),
 (361, 2, 68, 439),
 (361, 2, 68, 247),
 (361, 2, 363, 68),
 (361, 2, 68, 319),
 (361, 2, 235, 68),
 (361, 2, 299, 68),
 (361, 2, 435, 68),
 (361, 2, 68, 231),
 (361, 2, 123, 68),
 (361, 2, 68, 111),
 (361, 2, 68, 309),
 (361, 2, 68, 279),
 (361, 2, 323, 68),
 (361, 2, 161, 68),
 (361, 2, 441, 68),
 (361, 137, 2, 68),
 (361, 2, 68, 63),
 (361, 290, 2, 279),
 (2, 361, 290, 475),
 (361, 2, 475, 414),
 (361, 2, 414, 279),
 (249, 210, 2, 361),
 (2, 361, 458, 379),
 (361, 458, 2, 111),
 (361, 2, 227, 228),
 (2, 361, 74, 131),
 (361, 74, 2, 493),
 (361, 74, 2, 209),
 (361, 194, 2, 77),
 (361, 242, 2, 263),
 (361, 314, 2, 8

In [62]:
# ejemplo para ver quienes son los usuarios involucrados
for _, subgraph in get_all_matches(rdd_graph, "(x,0,y), (z,0,w)", more_info=True).collect():
    print(load_usernames(subgraph))


Se truncaron las últimas líneas 5000 del resultado de transmisión.
[('Michael Hunger', 'INTERACTS_WITH', 'davefauth'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'Excalidraw'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'Firefox 🔥'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'hicetnunc2000'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'widged'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'RAS KNII'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'Cloud Foundry'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'yusuf canbaz'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Hunger', 'INTERACTS_WITH', 'CEU Department